In [0]:
%sql
USE CATALOG public_sector;
USE SCHEMA predictive_maintenance_navy;

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW test AS
WITH s AS (
  SELECT *
  FROM sensor_silver
  LIMIT 100
)
SELECT 
  s.*,
  ai_query(
    endpoint => 'navy_predictive_maintenance',
    request => struct(
      hourly_timestamp,
      avg_energy,
      std_sensor_A,
      std_sensor_B,
      std_sensor_C,
      std_sensor_D,
      std_sensor_E,
      std_sensor_F,
      percentiles_sensor_A,
      percentiles_sensor_B,
      percentiles_sensor_C,
      percentiles_sensor_D,
      percentiles_sensor_E,
      percentiles_sensor_F
    ),
    returnType => 'STRING'
  ) AS prediction
FROM s

In [0]:
sensor_bronze_df = spark.read.table("public_sector.predictive_maintenance_navy_test.sensor_bronze")
test_df = spark.table("test")
joined_df = sensor_bronze_df.join(
    test_df,
    on="turbine_id"
)
result_df = joined_df.select("turbine_id", "abnormal_sensor", "prediction")
display(result_df)

In [0]:
import mlflow


In [0]:
import mlflow.pyfunc

model_uri = "models:/public_sector.predictive_maintenance_navy.predictive_maintenance_model/3"
model = mlflow.pyfunc.load_model(model_uri)

# Example: predictions = model.predict(your_dataframe)

df = spark.read.table("public_sector.predictive_maintenance_navy_test.turbine_training_dataset").limit(100)
labels = df.select("turbine_id", "abnormal_sensor")
predictions = model.predict(df.toPandas().drop('abnormal_sensor', axis=1))

display(spark.createDataFrame(predictions))

In [0]:
display(labels)

In [0]:
import os
import requests
import numpy as np
import pandas as pd
import json

def create_tf_serving_json(data):
    return {'inputs': {name: data[name].tolist() for name in data.keys()} if isinstance(data, dict) else data.tolist()}

def score_model(dataset):
    url = 'https://e2-demo-field-eng.cloud.databricks.com/serving-endpoints/navy_predictive_maintenance/invocations'
    headers = {'Authorization': f'Bearer {os.environ.get("DATABRICKS_TOKEN")}', 'Content-Type': 'application/json'}
    ds_dict = {'dataframe_split': dataset.to_dict(orient='split')} if isinstance(dataset, pd.DataFrame) else create_tf_serving_json(dataset)
    data_json = json.dumps(ds_dict, allow_nan=True)
    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    return response.json()

In [0]:
import pandas as pd

def score_model(dataset):
    # Convert pandas.Timestamp to string
    if isinstance(dataset, pd.DataFrame):
        dataset = dataset.applymap(
            lambda x: x.isoformat() if isinstance(x, pd.Timestamp) else x
        )
        ds_dict = {'dataframe_split': dataset.to_dict(orient='split')}
    else:
        ds_dict = create_tf_serving_json(dataset)
    data_json = json.dumps(ds_dict, allow_nan=True)
    response = requests.request(
        method='POST',
        headers={'Authorization': f'Bearer {os.environ.get("DATABRICKS_TOKEN")}', 'Content-Type': 'application/json'},
        url=url,
        data=data_json
    )
    if response.status_code != 200:
        raise Exception(f"Request failed: {response.text}")
    return response.json()

In [0]:
preds = score_model(df.toPandas())